# 🧠🤖 Treinamento de Redes LSTM para Classificação

- **Deadline**: 24/08/2025
- **Entrega**: O trabalho deve ser entregue via sistema Testr.
- **Pontuação**: 50% da nota do T2 (+1 ponto extra).
- O trabalho deve ser realizado individualmente.



## Especificação


### Contexto

O trabalho consiste em realizar o treinamento de redes LSTM usando a base de dados [BBC News Archive disponível no kaggle](https://www.kaggle.com/datasets/hgultekin/bbcnewsarchive?select=bbc-news-data.csv). Esta base de dados contém 2.225 textos publicados no site de notícias da BBC news entre 2004-2005. Cada notícia foi classificada como sendo de um dos seguintes assuntos: business (negócios), entertainment (entretenimento), politics (política), sport (esportes), tech (tecnologia).

O objetivo do trabalho é treinar uma rede neural capaz de identificar o tema de um texto. 


### Implementação 

- Use o notebook de classificação de sentimentos como ponto de partida.
- use a biblioteca `kagglehub` para fazer o download do dataset no colab.
- Um dos modelos de *word embeddings* disponíveis na biblioteca `gensim` deve ser utilizado para mapear palavras em vetores. 
- Use o tipo `nn.LSTM` disponível no `pytorch` (não é necessário implementar a camada LSTM do zero).
- Os dados devem ser divididos em treino, validação e teste. Use o conjunto de validação para ajustar hiperparâmetros e para selecionar o modelo com melhor generalização. Avalie o modelo resultante usando o conjunto de teste apenas ao final. 
- Você pode optar por cortar os textos em um tamanho máximo (e.g., 100 palavras), como fizemos no notebook, para que os testes não demorem muito.
- Use o ambiente de `GPU` do colab para evitar que o treinamento demore excessivamente.
- Durante o desenvolvimento, é uma boa idéia usar um subconjunto (e.g., 10%) das notícias para que os testes sejam mais rápidos. Quando tudo estiver correto, faça o treinamento com a base completa.
- Deve ser plotado o gráfico mostrando a evolução da função de perda nos conjuntos de treino e validação. 
- Devem ser mostradas as métricas geradas pela função `classification_report` da biblioteca scikit-learn e a matriz de confusão para o conjunto de teste. 
- Faça alguns testes qualitativos com textos escritos com você (não use textos da base de dados).
- Discuta brevemente os resultados quantitativos e qualitativos (1-2 parágrafos, no máximo).



### Pontos Extras

Receberá um ponto extra, o aluno que:
- Utilizar um LLM baseado em Transformer pré-treinado (e.g., [BERT](https://medium.com/@davidlfliang/intro-getting-started-with-text-embeddings-using-bert-9f8c3b98dee6)) para mapear as notícias em *embeddings*.
- Utilizar uma rede Multilayer Perceptron para classificar os *embeddings*. 
- Comparar a performance desta solução com a LSTM. 

⚠️**IMPORTANTE**⚠️
- Não é necessário (nem recomendável considerando o prazo) tentar realizar *fine-tuning* do LLM pré-treinado.
- Estes modelos são SUPER-ULTRA-MASTER-BLASTER lentos na CPU. Use o ambiente de GPU do colab para evitar ficar 20h esperando para transformar os textos em *embeddings*.
- Salve os embeddings depois da geração para evitar ter que gerá-los novamente. Quando necessário, faça upload do arquivo novamente para o colab.

In [ ]:
# Instalação de bibliotecas necessárias (executar apenas uma vez)
!pip install -q kagglehub gensim torch torchvision torchaudio sklearn

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from gensim.models import KeyedVectors
from gensim import downloader as api
import re
import string

In [ ]:
# Download do dataset BBC News Archive usando kagglehub
# Será feito download e extração do arquivo CSV. Pode demorar alguns minutos.
import kagglehub

dataset_path = kagglehub.download_dataset("hgultekin/bbcnewsarchive", download_method="http", force_download=False)

# Carregar o arquivo CSV em um DataFrame
import os
csv_path = None
# Procura pelo arquivo CSV dentro do diretório baixado
for root, dirs, files in os.walk(dataset_path):
    for fname in files:
        if fname.endswith('.csv'):
            csv_path = os.path.join(root, fname)
            break
    if csv_path:
        break

assert csv_path is not None, "Arquivo CSV não encontrado no dataset"

# Carrega o DataFrame
bbc_df = pd.read_csv(csv_path)
print(bbc_df.head())

In [ ]:
# Função de limpeza de texto: remove pontuações, coloca em minúsculas e quebra em tokens

def clean_text(text):
    # remove pontuação
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    # remove dígitos
    text = re.sub(r'\d+', '', text)
    tokens = text.split()
    return tokens

# Aplica limpeza a todas as notícias
bbc_df['tokens'] = bbc_df['content'].apply(clean_text)

# Codifica rótulos (categorias) como inteiros
label_encoder = LabelEncoder()
bbc_df['label_id'] = label_encoder.fit_transform(bbc_df['category'])

# Divide o conjunto em treino, validação e teste
train_df, test_df = train_test_split(bbc_df, test_size=0.2, random_state=42, stratify=bbc_df['label_id'])
train_df, val_df  = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['label_id'])

print(f"Total de amostras: {len(bbc_df)}, Treino: {len(train_df)}, Validação: {len(val_df)}, Teste: {len(test_df)}")

In [ ]:
# Carrega embeddings pré-treinados do Gensim.
# Existem diversos modelos disponíveis; aqui usamos glove-wiki-gigaword-100 (100 dimensões).
embedding_model = api.load('glove-wiki-gigaword-100')
embedding_dim = embedding_model.vector_size

# Constroi um vocabulário apenas com as palavras presentes nos dados de treino e existentes no modelo de embeddings
vocab = {'<PAD>': 0, '<UNK>': 1}
embedding_matrix = [np.zeros(embedding_dim), np.zeros(embedding_dim)]  # índices 0 e 1 para PAD e UNK

for tokens in train_df['tokens']:
    for token in tokens:
        if token not in vocab and token in embedding_model:
            vocab[token] = len(vocab)
            embedding_matrix.append(embedding_model[token])

embedding_matrix = np.array(embedding_matrix)

print(f"Tamanho do vocabulário: {len(vocab)}")

In [ ]:
# Comprimento máximo das sequências (trunca ou faz padding para esse tamanho)
max_seq_length = 100

# Função que converte uma lista de tokens em índices do vocabulário
def tokens_to_indices(tokens, vocab, max_len):
    indices = []
    for token in tokens:
        if token in vocab:
            indices.append(vocab[token])
        else:
            indices.append(vocab['<UNK>'])
    # Truncar ou fazer padding
    if len(indices) < max_len:
        indices += [vocab['<PAD>']] * (max_len - len(indices))
    else:
        indices = indices[:max_len]
    return indices

# Dataset personalizado
class NewsDataset(Dataset):
    def __init__(self, df, vocab, max_len):
        self.df = df.reset_index(drop=True)
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        tokens = self.df.loc[idx, 'tokens']
        label = self.df.loc[idx, 'label_id']
        indices = tokens_to_indices(tokens, self.vocab, self.max_len)
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Cria instâncias dos datasets e DataLoaders
train_dataset = NewsDataset(train_df, vocab, max_seq_length)
val_dataset   = NewsDataset(val_df, vocab, max_seq_length)
test_dataset  = NewsDataset(test_df, vocab, max_seq_length)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, n_layers=1, bidirectional=False, dropout=0.5):
        super().__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        # Carrega pesos pré-treinados e congela para não treinar
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32), requires_grad=False)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                            num_layers=n_layers, batch_first=True,
                            bidirectional=bidirectional, dropout=dropout if n_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (hidden, cell) = self.lstm(embedded)
        # Pega o último estado oculto
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1,:,:]
        output = self.dropout(hidden)
        out = self.fc(output)
        return out

# Hiperparâmetros do modelo
hidden_dim = 128
output_dim = len(label_encoder.classes_)
n_layers = 1
bidirectional = True
model = LSTMClassifier(embedding_matrix, hidden_dim, output_dim, n_layers, bidirectional, dropout=0.5)

# Define critério de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Move para GPU se disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * inputs.size(0)
        preds = torch.argmax(outputs, dim=1)
        epoch_acc += (preds == labels).sum().item()
        total += inputs.size(0)
    return epoch_loss / total, epoch_acc / total


def evaluate(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            epoch_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(outputs, dim=1)
            epoch_acc += (preds == labels).sum().item()
            total += inputs.size(0)
    return epoch_loss / total, epoch_acc / total

In [ ]:
num_epochs = 10
best_val_acc = 0
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} - Treino Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    # Salva o melhor modelo com base na acurácia de validação
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_lstm_model.pt')
        print("
Melhor modelo salvo com acurácia de validação:", best_val_acc)

In [ ]:
# Carrega o melhor modelo salvo
test_model = LSTMClassifier(embedding_matrix, hidden_dim, output_dim, n_layers, bidirectional, dropout=0.5)
test_model.load_state_dict(torch.load('best_lstm_model.pt'))
test_model = test_model.to(device)

# Avalia no conjunto de teste
test_loss, test_acc = evaluate(test_model, test_loader, criterion)
print(f"Desempenho no conjunto de teste - Loss: {test_loss:.4f}, Acurácia: {test_acc:.4f}")